In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import numpy as np
import pandas as pd
import haversine as hs
import re
import csv

In [11]:
driver = webdriver.Chrome()

site = 'https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa?distanceRadius=0&page=1&limit=36&market=ALL&locations=%5Bcities_6-26%5D&viewType=listing'
driver.get(site)
sleep(1)

# Akceptacja plików cookie na stronie
driver.find_element(By.XPATH, '//button[text()="Akceptuję"]').click()
sleep(1)

# wyszukiwanie linków do ogłoszeń na 1 stronie
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
sleep(1)
ts=driver.find_elements(By.XPATH, '//a[@data-cy="listing-item-link"]')

announcements_links = []
for t in ts:
    announcements_links.append(t.get_attribute('href'))

# pętla na przechodzenie do dalczych stron i zbieranie tam linków - duży sleep, bo java script na stornie nie chciał się ławdować i scraper wariował
for i in range(2,172):
    try:
        driver.get(f'https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa?distanceRadius=0&page={i}&limit=36&market=ALL&locations=%5Bcities_6-26%5D&viewType=listing')
        sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        ts=driver.find_elements(By.XPATH, '//a[@data-cy="listing-item-link"]')
        for t in ts:
            announcements_links.append(t.get_attribute('href'))
    except:
        pass

# usuwanie duplikatów linków do ogłoszeń
announcements_links = list(dict.fromkeys(announcements_links))

# funkcja do wcyiągania cyfr i zmiany separatora decymalnego
def get_numbers(string):
    extract = ""
    for s in string:
        if s.isdigit() or s == ",":
            extract = extract + s
    return(extract.replace(',', '.', 1))

# przygotowywanie list dla wartości z ogłoszeń
cena_wynajmu = []
powierzchnia = []
lokalizacja = []
lokalizacja2 = []
liczba_pokoi = []
pietro = []
wysokosc_budynku = []
rodzaj_zabudowy = []
wyposazenie = []
rok_budowy = []
czy_winda = []
herf = []
latitude = []
longitude = []

# pętla, która zbiera wartości z ogłoszeń
#3
for i in range(0,len(announcements_links)):
    try:
        new_site = announcements_links[i]
        driver.get(new_site)
        sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        loc_map = driver.find_element(By.ID, 'map')
        loc_map.location_once_scrolled_into_view
        sleep(1)
        try:
            location_container = driver.find_element(By.XPATH, '//a[@aria-label="Adres"]').text
            rent_container = driver.find_element(By.XPATH, '//strong[@aria-label="Cena"]').text
            area_container = driver.find_element(By.XPATH, '//div[@data-testid="table-value-area"]').text
            room_num_container = driver.find_element(By.XPATH, '//div[@data-testid="table-value-rooms_num"]').text
            floor_container = driver.find_element(By.XPATH, '//div[@data-testid="table-value-floor"]').text
            building_type_container = driver.find_element(By.XPATH, '//div[@data-testid="table-value-building_type"]').text
            furnishings_container = driver.find_element(By.XPATH, '//div[@data-testid="table-value-equipment_types"]').text
            year_of_construction_container = driver.find_element(By.XPATH, '//div[@data-testid="table-value-build_year"]').text
            elevator_container = driver.find_element(By.XPATH, '//div[@data-testid="table-value-lift"]').text
            
            map_link_elem = driver.find_elements(By.XPATH, '//a[@title="Pokaż ten obszar w Mapach Google (otwiera się w nowym oknie)"]')
            map_link = [elem.get_attribute('href') for elem in map_link_elem]
            latitude_container = re.search('ll=(.*),',str(map_link[0])).group(1)
            longitude_container = re.search(',(.*)&z',str(map_link[0])).group(1)

            herf.append(announcements_links[i])
            cena_wynajmu.append(get_numbers(rent_container))
            powierzchnia.append(get_numbers(area_container)[:-1])
            lokalizacja.append(location_container.split(", ")[1])
            lokalizacja2.append(location_container.split(", ")[0])
            liczba_pokoi.append(get_numbers(room_num_container))
            
            latitude.append(latitude_container)
            longitude.append(longitude_container)
            
            rodzaj_zabudowy.append(building_type_container.replace('''Rodzaj zabudowy
''', '', 1))
            wyposazenie.append(furnishings_container.replace('''Wyposażenie
''', '', 1))
            rok_budowy.append(get_numbers(year_of_construction_container))
            czy_winda.append(elevator_container.replace('''Winda
''', '', 1))
            if "/" in floor_container:
                wysokosc_budynku.append(floor_container.split("/")[1])
                if 'parter' in floor_container.split("/")[0]:
                    pietro.append('0')
                else:
                    pietro.append(get_numbers(floor_container.split("/")[0]))
            else:
                pietro.append(get_numbers(floor_container))
                wysokosc_budynku.append(get_numbers(floor_container))
        except:
            pass
    except:
        pass

# tworzenie dataframe'u z list

df = pd.DataFrame({
    "link": herf,
    "cena_wynajmu": cena_wynajmu,
    "powierzchnia": powierzchnia,
    "lokalizacja": lokalizacja,
    "lokalizacja2": lokalizacja2,
    "liczba_pokoi": liczba_pokoi,
    "rodzaj_zabudowy": rodzaj_zabudowy,
    "wyposazenie": wyposazenie,
    "rok_budowy": rok_budowy,
    "czy_winda": czy_winda,
    "wysokosc_budynku": wysokosc_budynku,
    "pietro": pietro,
    "latitude":latitude,
    "longitude":longitude
})

# Dalej; uzupełnienie datafram'u o brakujace informacje, których nie dało się wyciągnąć z samych ogłoszeń

####### klima

ogloszenia_z_klimatyacja = []

# pętla na dalesze strony
for i in range(1,30):
    try:
        driver.get(f'https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa?distanceRadius=0&page={i}&limit=36&market=ALL&locations=%5Bcities_6-26%5D&media=%5B%5D&extras=%5BAIR_CONDITIONING%5D&viewType=listing')
        sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        ts=driver.find_elements(By.XPATH, '//a[@data-cy="listing-item-link"]')
        for t in range(3,39):
            try:
                ogloszenia_z_klimatyacja.append(ts[t].get_attribute('href'))
            except IndexError:
                break
    except:
        pass

# usuwanie duplikatów
ogloszenia_z_klimatyacja = list(dict.fromkeys(ogloszenia_z_klimatyacja))

#dodanie nowej kolumny do df, kóra przyjmuje wartość True jeśli ogłoszenie z klimatyzacją dopasowuje się do linku ogłoszenia w dfreimie
df['czy_klimatyzacja'] = df['link'].isin(ogloszenia_z_klimatyacja)

#zmiana wartości z boolean na int
df['czy_klimatyzacja'] = df['czy_klimatyzacja'].astype(int)

#zapisanie df'u jako plik csv
df.to_csv('C:/Users/kacpe/Desktop/Projects/R_project/otodom_scrap.csv',
          na_rep='NaN',
          index=False,
          encoding='utf-8'
)

In [85]:
df = pd.read_csv('C:/Users/kacpe/Desktop/Projects/R_project/otodom_scrap.csv')

rename_dict = {'cena_wynajmu'     : 'rent_price',
               'powierzchnia'     : 'area',
               'liczba_pokoi'     : 'rooms_num',
               'rodzaj_zabudowy'  : 'building_type',
               'rok_budowy'       : 'construction_year',
               'czy_winda'        : 'elevator',
               'wysokosc_budynku' : 'building_height',
               'pietro'           : 'floor',
               'czy_klimatyzacja' : 'air_conditioning',
               'wyposazenie'      : 'equipment',
               'lokalizacja'      : 'location_1',
               'lokalizacja2'     : 'location_2'
               }
df.rename(columns=rename_dict, inplace=True)

df['dishwasher'] = df['equipment'].str.contains('zmywarka')
df['tv']          = df['equipment'].str.contains('telewizor')
df['elevator']    = df['elevator'].str.contains('tak')

int_cols   = ['rent_price', 'rooms_num', 'construction_year', 'elevator', 'building_height', 'floor', 'air_conditioning', 'dishwasher', 'tv']
float_cols = ['area', 'latitude','longitude']

data = {'Station': ['Plac Wilsona', 'Wilanowska', 'Centrum', 'Stokłosy', 'Wierzbno',
                   'Wilanowska', 'Pole Mokotowskie', 'Swiętokrzyska', 'Plac Wilsona',
                   'Ratusz', 'Marymont', 'Imielin', 'Ratusz', 'Wierzbno', 'Kabaty',
                   'Ursynów', 'Natolin', 'Bemowo', 'Ulrychów','Księcia Janusza',
                   'Młynów', 'Płocka' , 'Rondo Daszyńskiego', 'Rondo ONZ', 'Nowy Świat Uniwersytet',
                   'Centrum Nauki Kopernik','Stadion Narodowy','Wileński','Szwedzka',
                   'Targówek+Mieszkaniowy', 'Zacisze', 'Kondratowicza', 'Bródno'],
        'Latitude': [52.26921, 52.17954, 52.23055, 52.15583, 52.18889, 52.18098, 52.2079,
                     52.23499, 52.26902, 52.2441, 52.27178, 52.14952, 52.24476, 52.18932,
                     52.13109, 52.16217, 52.14033, 52.2393879, 52.2402211, 52.238989,
                     52.2370368, 52.2331322, 52.2302503, 52.2326673, 52.2372522,
                     52.2405898, 52.2468026, 52.2552165, 52.2658294,
                     52.278611, 52.278611, 52.2844033, 52.2844033],
        'Longitude': [20.98417, 21.02511, 21.01091, 21.0347, 21.01675, 21.02364, 21.00837,
                      21.00835, 20.985, 21.00174, 20.97232, 21.04543, 21.00082, 21.01726,
                      21.06604, 21.02858, 21.05776, 20.9175868, 20.9286984, 20.9413589,
                      20.9582514, 20.9672229, 20.9812049, 20.9943419, 21.0163951,
                      21.029495, 21.0437104, 21.0373375, 21.0477777,
                      21.0478345, 21.0478345, 21.0489269, 21.0489269]}

metro_stations = pd.DataFrame(data)

def calc_distance(lat : float, lon : float):
    dist_lst = []
    for index, row in metro_stations.iterrows():
        loc1=(row['Latitude'],row['Longitude'])
        loc2=(lat,lon)
        dist_lst.append(hs.haversine(loc1,loc2))
    return round(min(dist_lst)*1000,0)

df['subway_distance'] = [calc_distance(lat, lon) for lat, lon in zip(df['latitude'], df['longitude'])]
df['center_distance'] = [round(hs.haversine((lat, lon),(52.2498984, 21.0537275)),2) for lat, lon in zip(df['latitude'], df['longitude'])]

csv_file_path = 'districts_map.csv'
districts = {}
with open(csv_file_path, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        location, district = row
        districts[location] = district

df['district'] = df['location_1'].apply(lambda x: districts.get(x))
df.loc[[2966], 'district'] = 'Ursynów'

center_districts = ['Żoliborz', 'Wola', 'Ochota', 'Mokotów', 'Śródmieście', 'Praga-Południe', 'Praga-Północ']

def center_or_outlying(district):
    return 1 if district in center_districts else 0

df['center'] = df['district'].apply(center_or_outlying)
df['outlying'] = 1 - df['center']

def replace_characters(s):
    replacements = {
        'ł': 'l',
        'Ł': 'L',
        'ą': 'a',
        'Ą': 'A',
        'ć': 'c',
        'Ć': 'C',
        'ę': 'e',
        'Ę': 'E',
        'ń': 'n',
        'Ń': 'N',
        'ó': 'o',
        'Ó': 'O',
        'ś': 's',
        'Ś': 'S',
        'ż': 'z',
        'Ż': 'Z',
        'ź': 'z',
        'Ź': 'Z',
        '-': '_'
    }
    for search, replace in replacements.items():
        s = s.replace(search, replace)
    return s

df['District'] = df['district'].apply(replace_characters)

df = pd.get_dummies(df, columns=['District'])



building_type_rename = {
    'apartamentowiec' : 'apartment',
    'kamienica' : 'tenement',
    'blok' : 'block of flats'
}

df['building_type'] = df['building_type'].apply(lambda x: building_type_rename.get(x))
df = df.dropna(subset='building_type')

df['Building_type'] = df['building_type'].apply(replace_characters)

df = pd.get_dummies(df, columns=['Building_type'])

df.replace({False: 0, True: 1}, inplace=True)

In [87]:
df.to_csv('otodom_final.csv', index=False, encoding='utf-8')

In [88]:
df.columns

Index(['link', 'rent_price', 'area', 'location_1', 'location_2', 'rooms_num',
       'building_type', 'equipment', 'construction_year', 'elevator',
       'building_height', 'floor', 'latitude', 'longitude', 'air_conditioning',
       'dishwasher', 'tv', 'subway_distance', 'center_distance', 'district',
       'center', 'outlying', 'District_Adjacent areas', 'District_Bemowo',
       'District_Bialoleka', 'District_Bielany', 'District_Mokotow',
       'District_Ochota', 'District_Praga_Polnoc', 'District_Praga_Poludnie',
       'District_Rembertow', 'District_Srodmiescie', 'District_Targowek',
       'District_Ursus', 'District_Ursynow', 'District_Wawer',
       'District_Wesola', 'District_Wilanow', 'District_Wlochy',
       'District_Wola', 'District_Zoliborz', 'Building_type_apartment',
       'Building_type_block of flats', 'Building_type_tenement'],
      dtype='object')